In [1]:
import pandas as pd
import numpy as np
from env import get_db_url
import os
from sklearn.model_selection import train_test_split
import pandas as pd
from pydataset import data
from env import get_db_url
import pandas as pd
import numpy as np



## Remove any properties that are likely to be something other than single unit properties. (e.g. no duplexes, no land/lot, ...). There are multiple ways to estimate that a property is a single unit, and there is not a single "right" answer.

In [2]:
def new_zillow_data():
    """
    Fetches Zillow data from the database using SQL query.

    Returns:
    - df: DataFrame containing Zillow data
    """
   
    conn = get_db_url('zillow')

    query = '''
        SELECT *
    FROM properties_2017 AS p
    JOIN predictions_2017 AS pr ON p.parcelid = pr.parcelid
    LEFT JOIN airconditioningtype AS act ON p.airconditioningtypeid = act.airconditioningtypeid
    LEFT JOIN architecturalstyletype AS ast ON p.architecturalstyletypeid = ast.architecturalstyletypeid
    LEFT JOIN buildingclasstype AS bct ON p.buildingclasstypeid = bct.buildingclasstypeid
    LEFT JOIN heatingorsystemtype AS hot ON p.heatingorsystemtypeid = hot.heatingorsystemtypeid
    LEFT JOIN propertylandusetype AS plt ON p.propertylandusetypeid = plt.propertylandusetypeid
    LEFT JOIN storytype AS st ON p.storytypeid = st.storytypeid
    LEFT JOIN typeconstructiontype AS tct ON p.typeconstructiontypeid = tct.typeconstructiontypeid
    LEFT JOIN unique_properties AS up ON p.parcelid = up.parcelid
    WHERE p.unitcnt = 1;
    
    '''

    df = pd.read_sql(query, conn)
    return df

## Create a function that will drop rows or columns based on the percent of values that are missing: handle_missing_values(df, prop_required_column, prop_required_row).

In [3]:
def handle_missing_values(df, prop_required_column=0.5, prop_required_row=0.5):
    """
    Drops columns and rows from a DataFrame based on the percentage of missing values.
    
    Args:
        df (pandas.DataFrame): The DataFrame to handle missing values.
        prop_required_column (float, optional): The minimum proportion of non-null values required for columns (default is 0.5).
        prop_required_row (float, optional): The minimum proportion of non-null values required for rows (default is 0.5).
        
    Returns:
        pandas.DataFrame: The DataFrame with dropped rows and columns.
    """
    
    threshold_col = int(round(prop_required_column * len(df.index)))
    df = df.dropna(thresh=threshold_col, axis=1)
    
  
    threshold_row = int(round(prop_required_row * len(df.columns)))
    df = df.dropna(thresh=threshold_row)
    
    return df


In [4]:
def missing_value_summary(df):
    """
    Returns a DataFrame summarizing the count and percentage of missing values for each attribute in the input DataFrame.

    Args:
        df (pandas.DataFrame): The input DataFrame of observations and attributes.

    Returns:
        pandas.DataFrame: The summary DataFrame with count and percentage of missing values for each attribute.
    """
    missing_counts = df.isnull().sum()
    missing_percentages = (missing_counts / len(df)) * 100
    summary_df = pd.DataFrame({
        'Attribute': missing_counts.index,
        'Missing Count': missing_counts.values,
        'Missing Percentage': missing_percentages.values
    })
    return summary_df




## The input:

## A dataframe

In [5]:
df = new_zillow_data()

In [6]:
df

,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,buildingclassdesc,heatingorsystemtypeid,heatingorsystemdesc,propertylandusetypeid,propertylandusedesc,storytypeid,storydesc,typeconstructiontypeid,typeconstructiondesc,parcelid
0,2288172,12177905,NaN,None,None,3.0,4.0,NaN,8.0,3.0,...,None,2.0,Central,261,Single Family Residential,None,None,None,None,12177905
1,1970746,10887214,1.0,None,None,3.0,3.0,NaN,8.0,3.0,...,None,2.0,Central,266,Condominium,None,None,None,None,10887214
2,781532,12095076,1.0,None,None,3.0,4.0,NaN,9.0,3.0,...,None,2.0,Central,261,Single Family Residential,None,None,None,None,12095076
3,870991,12069064,NaN,None,None,1.0,2.0,NaN,5.0,1.0,...,None,NaN,None,261,Single Family Residential,None,None,None,None,12069064
4,1246926,12790562,NaN,None,None,3.0,4.0,NaN,9.0,3.0,...,None,2.0,Central,261,Single Family Residential,None,None,None,None,12790562
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47409,2864704,10833991,1.0,None,None,3.0,3.0,NaN,8.0,3.0,...,None,2.0,Central,266,Condominium,None,None,None,None,10833991
47410,673515,11000655,NaN,None,None,2.0,2.0,NaN,6.0,2.0,...,None,2.0,Central,261,Single Family Residential,None,None,None,None,11000655
47411,1843709,12773139,1.0,None,None,1.0,3.0,NaN,4.0,1.0,...,None,2.0,Central,261,Single Family Residential,None,None,None,None,12773139
47412,1187175,12826780,NaN,None,None,2.0,3.0,NaN,6.0,2.0,...,None,2.0,Central,261,Single Family Residential,None,None,None,None,12826780


## A number between 0 and 1 that represents the proportion, for each column, of rows with non-missing values required to keep the column. i.e. if prop_required_column = .6, then you are requiring a column to have at least 60% of values not-NA (no more than 40% missing).

## A number between 0 and 1 that represents the proportion, for each row, of columns/variables with non-missing values required to keep the row. For example, if prop_required_row = .75, then you are requiring a row to have at least 75% of variables with a non-missing value (no more that 25% missing).

## The output:

## The dataframe with the columns and rows dropped as indicated. Be sure to drop the columns prior to the rows in your function.

## hint:

## Look up the dropna documentation.

In [7]:
    def handle_missing_values(df, prop_required_column=0.5, prop_required_row=0.5):
        """
        Drops columns and rows from a DataFrame based on the percentage of missing values.

        Args:
            df (pandas.DataFrame): The DataFrame to handle missing values.
            prop_required_column (float, optional): The minimum proportion of non-null values required for columns (default is 0.5).
            prop_required_row (float, optional): The minimum proportion of non-null values required for rows (default is 0.5).

        Returns:
            pandas.DataFrame: The DataFrame with dropped rows and columns.
        """

        threshold_col = int(round(prop_required_column * len(df.index)))
        df = df.dropna(thresh=threshold_col, axis=1)


        threshold_row = int(round(prop_required_row * len(df.columns)))
        df = df.dropna(thresh=threshold_row)

        return df


## You will want to compute a threshold from your input values (prop_required) and total number of rows or columns.

In [8]:
df = handle_missing_values(df, prop_required_column=0.5, prop_required_row=0.5)

In [9]:
df

,id,parcelid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,fullbathcnt,...,censustractandblock,id,parcelid,logerror,transactiondate,heatingorsystemtypeid,heatingorsystemdesc,propertylandusetypeid,propertylandusedesc,parcelid
0,2288172,12177905,3.0,4.0,8.0,3.0,2376.0,2376.0,6037.0,3.0,...,6.037300e+13,3,12177905,-0.103410,2017-01-01,2.0,Central,261,Single Family Residential,12177905
1,1970746,10887214,3.0,3.0,8.0,3.0,1312.0,1312.0,6037.0,3.0,...,6.037124e+13,4,10887214,0.006940,2017-01-01,2.0,Central,266,Condominium,10887214
2,781532,12095076,3.0,4.0,9.0,3.0,2962.0,2962.0,6037.0,3.0,...,6.037461e+13,6,12095076,-0.001011,2017-01-01,2.0,Central,261,Single Family Residential,12095076
3,870991,12069064,1.0,2.0,5.0,1.0,738.0,738.0,6037.0,1.0,...,6.037302e+13,7,12069064,0.101723,2017-01-01,NaN,None,261,Single Family Residential,12069064
4,1246926,12790562,3.0,4.0,9.0,3.0,3039.0,3039.0,6037.0,3.0,...,6.037500e+13,8,12790562,-0.040966,2017-01-02,2.0,Central,261,Single Family Residential,12790562
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47409,2864704,10833991,3.0,3.0,8.0,3.0,1741.0,1741.0,6037.0,3.0,...,6.037132e+13,77608,10833991,-0.002245,2017-09-20,2.0,Central,266,Condominium,10833991
47410,673515,11000655,2.0,2.0,6.0,2.0,1286.0,1286.0,6037.0,2.0,...,6.037101e+13,77609,11000655,0.020615,2017-09-20,2.0,Central,261,Single Family Residential,11000655
47411,1843709,12773139,1.0,3.0,4.0,1.0,1032.0,1032.0,6037.0,1.0,...,6.037434e+13,77611,12773139,0.037129,2017-09-21,2.0,Central,261,Single Family Residential,12773139
47412,1187175,12826780,2.0,3.0,6.0,2.0,1762.0,1762.0,6037.0,2.0,...,6.037503e+13,77612,12826780,0.007204,2017-09-25,2.0,Central,261,Single Family Residential,12826780


## Encapsulate your work inside of functions in a wrangle_zillow.py module.

In [10]:
def new_zillow_data():
    """
    Fetches Zillow data from the database using SQL query.

    Returns:
    - df: DataFrame containing Zillow data
    """
   
    conn = get_db_url('zillow')

    query = '''
            SELECT *
            FROM properties_2017 p
            JOIN predictions_2017 pr ON p.parcelid = pr.parcelid
            WHERE p.unitcnt = 1
            '''

    
    df = pd.read_sql(query, conn)
    return df




    def handle_missing_values(df, prop_required_column=0.5, prop_required_row=0.5):
        """
        Drops columns and rows from a DataFrame based on the percentage of missing values.

        Args:
            df (pandas.DataFrame): The DataFrame to handle missing values.
            prop_required_column (float, optional): The minimum proportion of non-null values required for columns (default is 0.5).
            prop_required_row (float, optional): The minimum proportion of non-null values required for rows (default is 0.5).

        Returns:
            pandas.DataFrame: The DataFrame with dropped rows and columns.
        """

        threshold_col = int(round(prop_required_column * len(df.index)))
        df = df.dropna(thresh=threshold_col, axis=1)


        threshold_row = int(round(prop_required_row * len(df.columns)))
        df = df.dropna(thresh=threshold_row)

        return df



In [14]:
def check_duplicates(df):
    """
    Checks for duplicates in a DataFrame.

    Args:
        df (pandas.DataFrame): The DataFrame to check for duplicates.

    Returns:
        bool: True if duplicates are found, False otherwise.
    """
    duplicates = df.duplicated('parcelid')
    if duplicates.any():
        return True
    else:
        return False

# Example usage
duplicates_exist = check_duplicates(df)

duplicates_exist

True

In [48]:

def get_most_recent_transactions(df):
    
    duplicates_df = df[df.duplicated(subset='parcelid', keep=False)]

    
    sorted_df = duplicates_df.sort_values(by='transactiondate', ascending=False)

    
    most_recent_transactions = sorted_df.drop_duplicates(subset='parcelid', keep='first')

    return most_recent_transactions




In [49]:
most_recent_transactions = get_most_recent_transactions(df)

In [50]:
most_recent_transactions

,id,parcelid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,fullbathcnt,...,censustractandblock,id,parcelid,logerror,transactiondate,heatingorsystemtypeid,heatingorsystemdesc,propertylandusetypeid,propertylandusedesc,parcelid
47413,2455685,13083743,2.0,3.0,4.0,2.0,1050.0,1050.0,6037.0,2.0,...,6.037408e+13,77613,13083743,-0.197755,2018-05-25,7.0,Floor/Wall,261,Single Family Residential,13083743
6523,1094342,11957553,1.0,2.0,3.0,1.0,832.0,832.0,6037.0,1.0,...,6.037185e+13,10543,11957553,-0.116322,2017-09-19,7.0,Floor/Wall,261,Single Family Residential,11957553
17523,1406301,11594130,2.0,2.0,9.0,2.0,1835.0,1835.0,6037.0,2.0,...,6.037701e+13,28596,11594130,-0.103051,2017-09-19,2.0,Central,266,Condominium,11594130
4338,1535169,12478591,3.0,4.0,8.0,3.0,2941.0,2941.0,6037.0,3.0,...,6.037554e+13,7091,12478591,0.039378,2017-09-18,2.0,Central,261,Single Family Residential,12478591
20721,1416119,10979425,3.0,5.0,6.0,3.0,1850.0,1850.0,6037.0,3.0,...,6.037106e+13,33837,10979425,-0.241779,2017-09-18,2.0,Central,261,Single Family Residential,10979425
3706,1948842,12982361,1.0,3.0,4.0,1.0,1115.0,1115.0,6037.0,1.0,...,6.037433e+13,6062,12982361,0.127325,2017-09-14,7.0,Floor/Wall,261,Single Family Residential,12982361
6241,2782611,12827519,3.0,4.0,6.0,3.0,2058.0,2058.0,6037.0,3.0,...,6.037503e+13,10123,12827519,-0.009423,2017-09-13,2.0,Central,261,Single Family Residential,12827519
39346,2876815,10984080,3.0,3.0,8.0,3.0,1810.0,1810.0,6037.0,3.0,...,6.037104e+13,64944,10984080,0.860596,2017-09-12,2.0,Central,266,Condominium,10984080
21606,1169331,12099888,2.0,2.0,8.0,2.0,2547.0,2547.0,6037.0,2.0,...,6.037464e+13,35386,12099888,-0.058501,2017-09-12,2.0,Central,261,Single Family Residential,12099888
21584,2176302,11106339,3.0,4.0,8.0,3.0,2182.0,2182.0,6037.0,3.0,...,6.037920e+13,35357,11106339,-0.140602,2017-09-08,2.0,Central,269,Planned Unit Development,11106339
